In [36]:
import sys

def fillEmptyNew(original_df,colnum,flag):
    # assumed: df is the dataframe to operate on,
    # colnum is the column number with missing values
    # flag = 0/1 is whether it is a classification or regression problem
    
    # copying the original dataframe
    df = original_df.copy()
    
    # testing for valid flag
    if(flag != 0 and flag != 1):
        print('Invalid input flag')
        sys.exit()
    # testing for valid column number
    number_of_columns = len(list(original_df))
    # testing for the existence of empty column values
    a = df.iloc[:, [colnum]].isnull()
    idx = []
    for col in a:
        i=0
        for c in a[col]:
            if(c == True):
                idx.append(i)
            i=i+1
    if(len(idx) == 0):
        print('No empty values for input column number')
        sys.exit()
    
    # now can start pre-processing:
    # This converts all columns with "object" variables (AKA string) into numbers, and creates a dictionary  
    char_cols = df.dtypes.pipe(lambda x: x[x == 'object']).index
    label_mapping = {}
    for c in char_cols:
        df[c], label_mapping[c] = pd.factorize(df[c])
        
    # for the sake of classifying/ predicting current column,
    # with other columns may having null values,
    # we will replace the other columns' null val
    for c in df:
        df[c] = df[c].fillna(df[c].mean())

    # although some issue with the above may arise if null:
    # re-set all null values to null
    df.iloc[idx,[colnum]] = np.nan
    # Accessing the rows without empty values at colnum
    df_complete = df.dropna()
    df_complete.shape
    # Accessing the rows with empty values at colnum
    df_empty = df.iloc[idx]
    df_empty.shape
    # Splitting complete rows into target/features
    features = df_complete.drop(df.columns[[colnum]], axis=1)
    target_variable = df_complete.iloc[:, [colnum]]
    # Splitting the rows with empty colnum into features and response (which is what we're predicting)
    features_empty = df_empty.drop(df.columns[[colnum]], axis=1)
    
    # now can start classifying/ predicting:
    y_new = []
    if(flag==0): # classification
        from sklearn.ensemble import RandomForestClassifier
        print('Classifying...')
        print()
        # Training random forest
        randFor = RandomForestClassifier(n_estimators = 20)
        randFor.fit(features, target_variable)
        # Accuracy on the training set set
        print('Training score: ',randFor.score(features, target_variable))
        # Set of "City Group" predictions for the rows with empty values 
        y_new = randFor.predict(features_empty)
    else: # prediction
        from sklearn.linear_model import Ridge
        print('Predicting...')
        print()
        # Ridge Regression
        ridgereg = Ridge(normalize=True)
        ridgereg.fit(features,target_variable)
        y_new = ridgereg.predict(features_empty)
    # add the new values to the dataframe
    df.iloc[idx,[colnum]] = y_new
    df[colname+'_synthesized'] = 0
    df[colname+'_synthesized'][idx] = 1
    print(df)

In [ ]:
import pandas as pd
import numpy as np

# Testing on every datatset implemented or given out during lecture this semester.
datasets = [
'data_modelling.csv',
'diabetesdata.csv',
'eeg.csv',
'Energy.csv',
'iris_classification.csv',
'test.csv',
'train.csv',
'yelp.csv']

dfs = [pandas.read_csv(i) for i in datasets]

# In addition to some datatsets already having NaN values
# Empty cells at these determinitistic randomly chosen locations
lst = [(1, 1), (0, 1), (1, 3), (1, 6), (4, 0), (4, 2), (4, 3), (4, 6), (5, 0), (5, 2), (5, 3), (5, 6), (8, 0)]
for count,df in enumerate(dfs):
    for i in lst:
            df.set_value(r, c, None)
    print("Commencing Empty Value Filling For " + datasets[count])      
    fillEmptyNew(df,1,1)